In [7]:
import os
import requests
import zipfile
from jarvis.db.figshare import data
from jarvis.core.atoms import Atoms

def download_raw_files(jid, raw_files_list, output_dir="jarvis_files"):
 
    os.makedirs(output_dir, exist_ok=True)

    for raw_file_entry in raw_files_list:
        try:
            parts = raw_file_entry.split(',')
            file_name = parts[1]  # Название файла
            file_url = parts[2]   # URL для скачивания
        except IndexError:
            print(f" Ошибка обработки строки: {raw_file_entry}")
            continue

        zip_path = os.path.join(output_dir, file_name)

        # Скачиваем архив
        print(f"🔄 Скачиваем файл {file_name} для {jid}: {file_url}")
        response = requests.get(file_url)
        if response.status_code == 200:
            with open(zip_path, 'wb') as f:
                f.write(response.content)
            print(f"  Файл сохранён: {zip_path}")
        else:
            print(f" Не удалось скачать {file_name}. HTTP статус: {response.status_code}")
            continue

        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(output_dir)
            print(f"  Архив {file_name} распакован в: {output_dir}")
        except zipfile.BadZipFile:
            print(f" Ошибка распаковки архива {file_name}.")


dft_2d_data = data('dft_2d')
material_data = next((m for m in dft_2d_data if m['formula'] == 'C'), None)

if material_data:
    jid = material_data['jid']
    raw_files_list = material_data.get('raw_files', [])
    if raw_files_list:
        print(f"🔗 Найден список ссылок для {jid}: {raw_files_list}")
        download_raw_files(jid, raw_files_list)
    else:
        print(f" У материала {jid} нет ссылок на raw_files.")
else:
    print("Материал с формулой 'C' не найден.")


Obtaining 2D dataset 1.1k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815705
Loading the zipfile...
Loading completed.
🔗 Найден список ссылок для JVASP-60368: ['STM,JVASP-60368_Positive.zip,https://ndownloader.figshare.com/files/24962048', 'STM,JVASP-60368_Negative.zip,https://ndownloader.figshare.com/files/25035113', 'OPT-LOPTICS,JVASP-60368.zip,https://ndownloader.figshare.com/files/24863711', 'DFT-SCF,JVASP-60368.zip,https://ndownloader.figshare.com/files/23474213']
🔄 Скачиваем файл JVASP-60368_Positive.zip для JVASP-60368: https://ndownloader.figshare.com/files/24962048
✅ Файл сохранён: jarvis_files/JVASP-60368_Positive.zip
✅ Архив JVASP-60368_Positive.zip распакован в: jarvis_files
🔄 Скачиваем файл JVASP-60368_Negative.zip для JVASP-60368: https://ndownloader.figshare.com/files/25035113
✅ Файл сохранён: jarvis_files/JVASP-60368_Negative.zip
✅ Архив JVASP-60368_Negative.zip распакован в: jarvis_files
🔄 Скачива

In [9]:
import numpy as np
from ase import Atoms
from scipy.spatial.distance import pdist, squareform

def generate_hcore(atoms: Atoms, z_values, alpha=1.0):
    """
    Генерирует матрицу ядра H_core на основе кинетической и потенциальной энергии.
    
    Parameters:
    atoms : Atoms
        Объект ASE с атомами.
    z_values : list
        Список зарядов атомов.
    alpha : float
        Весовой коэффициент для кинетической энергии.
    
    Returns:
    H_core : np.ndarray
        Матрица ядра.
    """
    num_atoms = len(atoms)
    positions = atoms.get_positions()
    distances = squareform(pdist(positions)) + np.eye(num_atoms)  # Расстояния, +1 на диагонали
    
    H_kin = alpha * np.exp(-distances**2)
    
    H_nuc = np.zeros((num_atoms, num_atoms))
    for i in range(num_atoms):
        for j in range(num_atoms):
            if i != j:
                H_nuc[i, j] = -z_values[i] * z_values[j] / distances[i, j]
   
    H_core = H_kin + H_nuc
    return H_core

atoms = Atoms(symbols='C2', positions=[[0, 0, 0], [1.4, 0, 0]], cell=[10, 10, 10], pbc=True)
z_values = [6, 6] 
H_core = generate_hcore(atoms, z_values)

print("Обновлённая матрица H_core:\n", H_core)

def generate_matrices(atoms: Atoms, alpha=0.5):
    """
    Генерирует матрицу перекрытия S и матрицу ядра H_core.
    
    Parameters:
    atoms : Atoms
        Атомы из ASE (координаты системы).
    alpha : float
        Параметр экспоненциального затухания для S.
    
    Returns:
    S : np.ndarray
        Матрица перекрытия.
    H_core : np.ndarray
        Матрица ядра.
    """
    num_atoms = len(atoms)
    positions = atoms.get_positions()

    distances = squareform(pdist(positions))

    S = np.exp(-alpha * distances**2)
    np.fill_diagonal(S, 1.0)  

    H_core = np.random.rand(num_atoms, num_atoms)
    H_core = (H_core + H_core.T) / 2  # Симметризуем матрицу

    return S, H_core

atoms = Atoms(symbols='C2', positions=[[0, 0, 0], [1.4, 0, 0]], cell=[10, 10, 10], pbc=True)
S, H_core = generate_matrices(atoms)

print("Матрица перекрытия S:\n", S)
print("Матрица ядра H_core:\n", H_core)

def hartree_fock_iteration(S, H_core, max_iter=100, tol=1e-6):
    """
    Итерационный процесс Хартри-Фока.
    
    Parameters:
    S : np.ndarray
        Матрица перекрытия.
    H_core : np.ndarray
        Матрица ядра.
    max_iter : int
        Максимальное количество итераций.
    tol : float
        Точность сходимости.
    
    Returns:
    energy : float
        Итоговая энергия TOTEN.
    """
    num_atoms = S.shape[0]

    # Начальная догадка для матрицы C
    C = np.eye(num_atoms)
    energy_prev = 0.0

    for iteration in range(max_iter):
        G = np.random.rand(num_atoms, num_atoms)
        G = (G + G.T) / 2

        F = H_core + G

        eigvals, eigvecs = np.linalg.eigh(F)

        energy = np.sum(eigvals)
        print(f"Итерация {iteration + 1}: Энергия = {energy:.6f} эВ")

        if abs(energy - energy_prev) < tol:
            print(f"Сходимость достигнута на итерации {iteration + 1}.")
            break

        energy_prev = energy

    return energy

TOTEN = hartree_fock_iteration(S, H_core)
print(f"Итоговая энергия TOTEN: {TOTEN:.6f} эВ")


Обновлённая матрица H_core:
 [[  0.36787944 -25.57342729]
 [-25.57342729   0.36787944]]
Матрица перекрытия S:
 [[1.        0.3753111]
 [0.3753111 1.       ]]
Матрица ядра H_core:
 [[0.57644331 0.63405141]
 [0.63405141 0.23455107]]
Итерация 1: Энергия = 1.513648 эВ
Итерация 2: Энергия = 1.354917 эВ
Итерация 3: Энергия = 2.409835 эВ
Итерация 4: Энергия = 1.430971 эВ
Итерация 5: Энергия = 1.875693 эВ
Итерация 6: Энергия = 2.224390 эВ
Итерация 7: Энергия = 1.875934 эВ
Итерация 8: Энергия = 1.353060 эВ
Итерация 9: Энергия = 1.893949 эВ
Итерация 10: Энергия = 1.861312 эВ
Итерация 11: Энергия = 1.897152 эВ
Итерация 12: Энергия = 1.606997 эВ
Итерация 13: Энергия = 2.357753 эВ
Итерация 14: Энергия = 2.084733 эВ
Итерация 15: Энергия = 2.211444 эВ
Итерация 16: Энергия = 1.678721 эВ
Итерация 17: Энергия = 2.500710 эВ
Итерация 18: Энергия = 1.898216 эВ
Итерация 19: Энергия = 2.083461 эВ
Итерация 20: Энергия = 2.087272 эВ
Итерация 21: Энергия = 1.831664 эВ
Итерация 22: Энергия = 2.477914 эВ
Итерация

In [11]:
import os
import requests
import zipfile
from jarvis.db.figshare import data
from jarvis.core.atoms import Atoms
from ase.io import read
import numpy as np
from scipy.spatial.distance import pdist, squareform

def download_raw_files(jid, raw_files_list, output_dir="jarvis_files"):
    """
    Скачивает архивы raw_files и распаковывает их.
    """
    os.makedirs(output_dir, exist_ok=True)

    for raw_file_entry in raw_files_list:
        try:
            parts = raw_file_entry.split(',')
            file_name = parts[1]
            file_url = parts[2]
        except IndexError:
            print(f" Ошибка обработки строки: {raw_file_entry}")
            continue

        zip_path = os.path.join(output_dir, file_name)
        print(f"🔄 Скачиваем файл {file_name} для {jid}: {file_url}")
        response = requests.get(file_url)
        if response.status_code == 200:
            with open(zip_path, 'wb') as f:
                f.write(response.content)
            print(f"  Файл сохранён: {zip_path}")
        else:
            print(f" Не удалось скачать {file_name}. HTTP статус: {response.status_code}")
            continue

        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(output_dir)
            print(f"  Архив {file_name} распакован в: {output_dir}")
        except zipfile.BadZipFile:
            print(f" Ошибка распаковки архива {file_name}.")

def parse_vasp_energy(output_dir):
    """
    Извлекает TOTEN из OUTCAR или других файлов VASP.
    """
    outcar_path = os.path.join(output_dir, "OUTCAR")
    if not os.path.exists(outcar_path):
        print(" Файл OUTCAR не найден.")
        return None
    
    with open(outcar_path, 'r') as f:
        lines = f.readlines()
    
    for line in lines:
        if "TOTEN" in line:
            try:
                return float(line.split()[-2])
            except ValueError:
                continue
    print(" TOTEN не найден в OUTCAR.")
    return None

def generate_hcore_advanced(atoms: Atoms, z_values, alpha=1.0, beta=1.0):
    num_atoms = len(atoms)
    positions = atoms.get_positions()
    distances = squareform(pdist(positions)) + np.eye(num_atoms)
    
    H_kin = alpha * np.exp(-distances**2)
  
    H_nuc = np.zeros((num_atoms, num_atoms))
    for i in range(num_atoms):
        for j in range(num_atoms):
            if i != j:
                H_nuc[i, j] = -z_values[i] * z_values[j] / distances[i, j]
    
    # Корреляция (условный вклад)
    H_corr = beta / distances
    np.fill_diagonal(H_corr, 0) 

    return H_kin + H_nuc + H_corr

def improve_two_electron_interaction(S, distances, gamma=1.0):

    num_atoms = S.shape[0]
    G = np.zeros((num_atoms, num_atoms))
    for i in range(num_atoms):
        for j in range(i + 1, num_atoms):
            interaction = gamma / distances[i, j]
            G[i, j] = interaction
            G[j, i] = interaction
    return G

def hartree_fock_iteration_advanced(S, H_core, distances, max_iter=100, tol=1e-6, gamma=1.0):

    num_atoms = S.shape[0]
    energy_prev = 0.0

    for iteration in range(max_iter):
        G = improve_two_electron_interaction(S, distances, gamma)

        F = H_core + G

        eigvals, _ = np.linalg.eigh(F)

        energy = np.sum(eigvals)
        print(f"Итерация {iteration + 1}: Энергия = {energy:.6f} эВ")

        if abs(energy - energy_prev) < tol:
            break
        energy_prev = energy
    return energy

z_values = [6] * len(atoms) 
H_core_advanced = generate_hcore_advanced(atoms, z_values)
distances = squareform(pdist(atoms.get_positions()))

TOTEN_advanced = hartree_fock_iteration_advanced(S, H_core_advanced, distances)
print(f"Энергия TOTEN (улучшенная модель Хартри-Фок): {TOTEN_advanced:.6f} эВ")

if material_data:
    jid = material_data['jid']
    raw_files_list = material_data.get('raw_files', [])
    output_dir = f"jarvis_files/{jid}"
    if raw_files_list:
        download_raw_files(jid, raw_files_list, output_dir)
        vasp_energy = parse_vasp_energy(output_dir)
        if vasp_energy is not None:
            print(f"Энергия TOTEN (VASP): {vasp_energy:.6f} эВ")
            
            poscar_path = os.path.join(output_dir, "POSCAR")
            atoms = read(poscar_path, format="vasp")
            
            z_values = [6] * len(atoms) 
            S, H_core = generate_hcore(atoms, z_values), generate_hcore(atoms, z_values)
            
            hf_energy = hartree_fock_iteration(S, H_core)
            print(f"Энергия TOTEN (Хартри-Фок): {hf_energy:.6f} эВ")
            
            error = abs(vasp_energy - hf_energy) / abs(vasp_energy) * 100
            print(f"Погрешность: {error:.2f}%")
            if error <= 10:
                print("  Погрешность удовлетворяет требованию (< 10%).")
            else:
                print(" Погрешность превышает 10%.")
        else:
            print(" Не удалось извлечь TOTEN из VASP.")
    else:
        print(f" У материала {jid} нет ссылок на raw_files.")
else:
    print(" Материал с формулой 'C' не найден.")



Итерация 1: Энергия = 0.735759 эВ
Итерация 2: Энергия = 0.735759 эВ
Энергия TOTEN (улучшенная модель Хартри-Фок): 0.735759 эВ
🔄 Скачиваем файл JVASP-60368_Positive.zip для JVASP-60368: https://ndownloader.figshare.com/files/24962048
✅ Файл сохранён: jarvis_files/JVASP-60368/JVASP-60368_Positive.zip
✅ Архив JVASP-60368_Positive.zip распакован в: jarvis_files/JVASP-60368
🔄 Скачиваем файл JVASP-60368_Negative.zip для JVASP-60368: https://ndownloader.figshare.com/files/25035113
✅ Файл сохранён: jarvis_files/JVASP-60368/JVASP-60368_Negative.zip
✅ Архив JVASP-60368_Negative.zip распакован в: jarvis_files/JVASP-60368
🔄 Скачиваем файл JVASP-60368.zip для JVASP-60368: https://ndownloader.figshare.com/files/24863711
✅ Файл сохранён: jarvis_files/JVASP-60368/JVASP-60368.zip
✅ Архив JVASP-60368.zip распакован в: jarvis_files/JVASP-60368
🔄 Скачиваем файл JVASP-60368.zip для JVASP-60368: https://ndownloader.figshare.com/files/23474213
✅ Файл сохранён: jarvis_files/JVASP-60368/JVASP-60368.zip
✅ Архив

In [20]:
import os
import requests
import zipfile
from jarvis.db.figshare import data
from ase.io import read
from scipy.spatial.distance import pdist, squareform
import numpy as np

# Download raw files
def download_raw_files(jid, raw_files_list, output_dir="jarvis_files"):
    os.makedirs(output_dir, exist_ok=True)
    for raw_file_entry in raw_files_list:
        try:
            parts = raw_file_entry.split(',')
            file_name = parts[1]
            file_url = parts[2]
        except IndexError:
            continue
        zip_path = os.path.join(output_dir, file_name)
        response = requests.get(file_url)
        if response.status_code == 200:
            with open(zip_path, 'wb') as f:
                f.write(response.content)
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(output_dir)

# Improved H_core generation
def generate_hcore_advanced(atoms, z_values, alpha=1.0, beta=1.0):
    num_atoms = len(atoms)
    positions = atoms.get_positions()
    distances = squareform(pdist(positions)) + np.eye(num_atoms)
    H_kin = alpha * np.exp(-distances**2)
    H_nuc = np.zeros((num_atoms, num_atoms))
    for i in range(num_atoms):
        for j in range(num_atoms):
            if i != j:
                H_nuc[i, j] = -z_values[i] * z_values[j] / distances[i, j]
    H_corr = beta / distances
    np.fill_diagonal(H_corr, 0)
    return H_kin + H_nuc + H_corr

def improve_two_electron_interaction(S, distances, gamma=1.0, beta=1.0):
    G = gamma * np.exp(-beta * distances)
    np.fill_diagonal(G, 0)
    return G

def hartree_fock_iteration(S, H_core, distances, max_iter=200, tol=1e-5, gamma=1.0, beta=1.0):
    num_atoms = S.shape[0]
    energy_prev = 0.0
    for iteration in range(max_iter):
        G = improve_two_electron_interaction(S, distances, gamma, beta)
        F = H_core + G
        eigvals, _ = np.linalg.eigh(F)
        energy = np.sum(eigvals)
        if abs(energy - energy_prev) < tol:
            break
        energy_prev = energy
    return energy

dft_2d_data = data('dft_2d')
results = []

for material in dft_2d_data[:10]:  # Limiting to 10 materials for demonstration
    jid = material['jid']
    raw_files_list = material.get('raw_files', [])
    output_dir = f"jarvis_files/{jid}"
    if raw_files_list:
        download_raw_files(jid, raw_files_list, output_dir)
        vasp_energy = material.get('optb88vdw_total_energy', None)
        if vasp_energy is not None:
            poscar_path = os.path.join(output_dir, "POSCAR")
            try:
                atoms = read(poscar_path, format="vasp")
                z_values = [6] * len(atoms) 
                distances = squareform(pdist(atoms.get_positions()))
                S = np.exp(-0.5 * distances**2)
                np.fill_diagonal(S, 1.0)
                H_core = generate_hcore_advanced(atoms, z_values, alpha=1.5, beta=1.0)
                hf_energy = hartree_fock_iteration(S, H_core, distances, gamma=1.2, beta=0.8)
                error = abs(vasp_energy - hf_energy) / abs(vasp_energy) * 100
                results.append({'jid': jid, 'vasp_energy': vasp_energy, 'hf_energy': hf_energy, 'error': error})
            except Exception as e:
                results.append({'jid': jid, 'vasp_energy': vasp_energy, 'hf_energy': None, 'error': str(e)})
        else:
            results.append({'jid': jid, 'vasp_energy': None, 'hf_energy': None, 'error': "Energy not found"})
    else:
        results.append({'jid': jid, 'vasp_energy': None, 'hf_energy': None, 'error': "Raw files not found"})

import pandas as pd

df_results = pd.DataFrame(results)
df_results.to_csv("hf_vs_vasp_results.csv", index=False)  
print(df_results.head()) 


Obtaining 2D dataset 1.1k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815705
Loading the zipfile...
Loading completed.
           jid  vasp_energy  hf_energy        error
0  JVASP-14441     -4.52094   1.655457   136.617550
1  JVASP-76308     -3.37838   4.414553   230.670715
2  JVASP-76515     -0.34456   4.414553  1381.214678
3  JVASP-13526     -2.77075   8.829107   418.654032
4  JVASP-27901     -3.75353   3.310915   188.208033
